# Importar la información desde un Dropbox u otro.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import math
import random
import os
import shutil

random.seed(30)

In [3]:
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/data.zip", "/content/data.zip")
shutil.copy("/content/drive/MyDrive/Taller Diplomado IA/Data Comerciales/processed/Comerciales.csv", "/content/Comerciales.csv")
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/2021_10_27_09_F10020.jpg  
  inflating: data/2021_10_27_09_F10080.jpg  
  inflating: data/2021_10_27_09_F10140.jpg  
  inflating: data/2021_10_27_09_F1020.jpg  
  inflating: data/2021_10_27_09_F10200.jpg  
  inflating: data/2021_10_27_09_F10260.jpg  
  inflating: data/2021_10_27_09_F10320.jpg  
  inflating: data/2021_10_27_09_F10380.jpg  
  inflating: data/2021_10_27_09_F10440.jpg  
  inflating: data/2021_10_27_09_F10500.jpg  
  inflating: data/2021_10_27_09_F10560.jpg  
  inflating: data/2021_10_27_09_F10620.jpg  
  inflating: data/2021_10_27_09_F10680.jpg  
  inflating: data/2021_10_27_09_F10740.jpg  
  inflating: data/2021_10_27_09_F1080.jpg  
  inflating: data/2021_10_27_09_F10800.jpg  
  inflating: data/2021_10_27_09_F10860.jpg  
  inflating: data/2021_10_27_09_F10920.jpg  
  inflating: data/2021_10_27_09_F10980.jpg  
  inflating: data/2021_10_27_09_F11040.jpg  
  inflating: data/2021_10_27_09_F11100.jpg  
  inflating: data/2

El dataset debe estar organizado en dos carpetas: train y test; además de dos archivos de texto csv con la información: video | frame | clase. 

Las dos clases son comercial (0) y noticias (1).

Se organiza el dataset con las dos subclases. 

# Transfer Learning

En este notebook resolveremos un problema real usando redes pre-entrenadas y transfer learning.


# La Solución

Primero creamos nuestras tres colecciones de datos: train, val y test. Los datos de train serán usados para entrenar el modelo, los datos de validación serán usados para probar el modelo durante entrenamiento, y los datos de test serán usados para evaluar el performance final del modelo.

Hacemos un poco de data engineering para crear nuestras colecciones. 

*   Datos de train: 70%
*   Datos de val: 10%
*   Datos de test: 20%

La partición se hace dentro de cada clase para asegurar representatividad en cada clase.



In [4]:
df = pd.read_csv('Comerciales.csv', header=0, sep=";")

In [5]:
df

,Nombre Final,Clase
0,2021_10_27_09_F60,Noticias
1,2021_10_27_09_F120,Noticias
2,2021_10_27_09_F180,Noticias
3,2021_10_27_09_F240,Noticias
4,2021_10_27_09_F300,Noticias
...,...,...
3559,2021_10_27_13_F120720,Comercial
3560,2021_10_27_13_F120780,Comercial
3561,2021_10_27_13_F120840,Comercial
3562,2021_10_27_13_F120900,Comercial


In [6]:
classesFinal = df["Clase"].unique()
#classesFinal = [cl.replace(' ', '_') for cl in classes]
print(classesFinal)

['Noticias' 'Comercial']


In [7]:
try:
  os.mkdir('dataset')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")

try:
  os.mkdir('dataset/train')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")

try:
  os.mkdir('dataset/test')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")

try:
  os.mkdir('dataset/val')
except OSError:
  print ("No se pudo crear folder dataset")
else:
  print ("Se creó folder dataset")

for cl in classesFinal:
  try:
    os.mkdir(os.path.join('dataset', "train", cl))
  except OSError:
    print (f"No se pudo crear folder train {cl}")
  else:
    print (f"Se creó folder train {cl}")
  
  try:
    os.mkdir(os.path.join('dataset', "test", cl))
  except OSError:
    print (f"No se pudo crear folder test {cl}")
  else:
    print (f"Se creó folder test {cl}")
  
  try:
    os.mkdir(os.path.join('dataset', "val", cl))
  except OSError:
    print (f"No se pudo crear folder val {cl}")
  else:
    print (f"Se creó folder val {cl}")


Se creó folder dataset
Se creó folder dataset
Se creó folder dataset
Se creó folder dataset
Se creó folder train Noticias
Se creó folder test Noticias
Se creó folder val Noticias
Se creó folder train Comercial
Se creó folder test Comercial
Se creó folder val Comercial


In [8]:
dataset = dict()

for index, cl in enumerate(classesFinal):
    grouped_data = df.groupby(by=["Clase"]).get_group(cl)["Nombre Final"].tolist()
    dataset[classesFinal[index]] = grouped_data

In [9]:
for k,v in dataset.items():
    print(f'Class: {k}, Length: {len(v)}')

    print(v)

Class: Noticias, Length: 3154
['2021_10_27_09_F60', '2021_10_27_09_F120', '2021_10_27_09_F180', '2021_10_27_09_F240', '2021_10_27_09_F300', '2021_10_27_09_F360', '2021_10_27_09_F420', '2021_10_27_09_F480', '2021_10_27_09_F540', '2021_10_27_09_F600', '2021_10_27_09_F660', '2021_10_27_09_F720', '2021_10_27_09_F780', '2021_10_27_09_F840', '2021_10_27_09_F900', '2021_10_27_09_F960', '2021_10_27_09_F1020', '2021_10_27_09_F1080', '2021_10_27_09_F1140', '2021_10_27_09_F1200', '2021_10_27_09_F1260', '2021_10_27_09_F5040', '2021_10_27_09_F5100', '2021_10_27_09_F5160', '2021_10_27_09_F5220', '2021_10_27_09_F5280', '2021_10_27_09_F5340', '2021_10_27_09_F5400', '2021_10_27_09_F5460', '2021_10_27_09_F5520', '2021_10_27_09_F5580', '2021_10_27_09_F5640', '2021_10_27_09_F5700', '2021_10_27_09_F5760', '2021_10_27_09_F5820', '2021_10_27_09_F5880', '2021_10_27_09_F5940', '2021_10_27_09_F6000', '2021_10_27_09_F6060', '2021_10_27_09_F6120', '2021_10_27_09_F6180', '2021_10_27_09_F6240', '2021_10_27_09_F6300

In [10]:
for k,v in dataset.items():
    valNumber = math.ceil(0.1 * len(v))
    testNumber = math.ceil(0.2 * len(v))
    trainNumber = len(v) - valNumber - testNumber

    random.shuffle(v)
    elemTrain = v[:trainNumber]
    elemVal = v[trainNumber:trainNumber+valNumber]
    elemTest = v[trainNumber+valNumber:]

    assert (valNumber + testNumber + trainNumber) == len(v)

    pathTrain = './dataset/train/'
    pathSource = './data/'

    #Copiar archivos de train
    for elem in elemTrain:
        shutil.copy(os.path.join(pathSource,elem+'.jpg'), os.path.join(pathTrain, k))
    
    pathTest = './dataset/test/'
    pathSource = './data/'

    #Copiar archivos de test
    for elem in elemTest:
        shutil.copy(os.path.join(pathSource,elem+'.jpg'), os.path.join(pathTest, k))
    
    pathVal = './dataset/val/'
    pathSource = './data/'

    #Copiar archivos de train
    for elem in elemVal:
        shutil.copy(os.path.join(pathSource,elem+'.jpg'), os.path.join(pathVal, k))

#Creación de Datasets

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy

pathDataset = 'dataset/'

train_dataset = torchvision.datasets.ImageFolder(pathDataset + 'train', 
                                                    transform = transforms.Compose([
                                                        transforms.RandomVerticalFlip(),
                                                        transforms.RandomHorizontalFlip(),
                                                        transforms.RandomResizedCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

val_dataset = torchvision.datasets.ImageFolder(pathDataset + 'val',
                                                    transform = transforms.Compose([ transforms.Resize(256),
                                                                    transforms.CenterCrop(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

test_dataset = torchvision.datasets.ImageFolder(pathDataset + 'test',
                                                    transform = transforms.Compose([ transforms.Resize(224),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std = [0.229, 0.224, 0.225])]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

class_names = train_dataset.classes

device = ('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds ==  labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print('Train Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        #Validation
        model.eval()
        running_loss = 0.0
        running_corrects = 0.0

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset)
        print('Val Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    print('Best accuracy: {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

def test_model(model, criterion):
  model.eval()
  running_loss = 0.0
  running_corrects = 0.0

  for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
            
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / len(test_dataset)
  epoch_acc = running_corrects / len(test_dataset)
  print('Test Loss: {:.4f}  Acc: {:.4f}'.format(epoch_loss, epoch_acc))


# Solución 1: fine-tuning

La técnica del fine-tuning se refiere a cambiar la arquitectura de una red conocida para resolver un nuevo problema, pero modificando todos los parámetros del modelo. Lo que se espera es que cada parámetro de la red neuronal aprenda a resolver el nuevo problema.



In [12]:
#~12 min
#Usamos un modelo pre-entrenado ResNet18 con dataset Imagenet
model_ft = models.resnet18(pretrained=True)

#Vamos a cambiar la última capa de la red. La red original fur entrenada con 
# 1000 clases. Ahora solo necesitamos 2 neuronas de salida.
# Cambiamos la arquitectura final de la red

num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 2)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

#La red original fue entrenada con SGD y un lr inicial de 0.1, el cual decrementaba cada vez que el 
# error se estancaba. Nosotros partimos con SGD y un lr bajo, dado que solo queremos tunear la red
# para el nuevo problema

optimizer = torch.optim.SGD(model_ft.parameters(), lr = 0.001, momentum=0.9)

# Empezamos con un número de épocas bajo. A más épocas podemos mejorar el performance
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=30)

#Guardamos la mejor red, en términos de accuracy de validación
torch.save(model_ft.state_dict(), 'resnet18_finetuned.pth')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch 0/29
----------
Train Loss: 0.2235  Acc: 0.9210
Val Loss: 0.0891  Acc: 0.9692
Epoch 1/29
----------
Train Loss: 0.1214  Acc: 0.9539
Val Loss: 0.0550  Acc: 0.9832
Epoch 2/29
----------
Train Loss: 0.0933  Acc: 0.9615
Val Loss: 0.0505  Acc: 0.9860
Epoch 3/29
----------
Train Loss: 0.0711  Acc: 0.9739
Val Loss: 0.0402  Acc: 0.9888
Epoch 4/29
----------
Train Loss: 0.0607  Acc: 0.9779
Val Loss: 0.0428  Acc: 0.9860
Epoch 5/29
----------
Train Loss: 0.0563  Acc: 0.9816
Val Loss: 0.0336  Acc: 0.9916
Epoch 6/29
----------
Train Loss: 0.0548  Acc: 0.9804
Val Loss: 0.0375  Acc: 0.9888
Epoch 7/29
----------
Train Loss: 0.0577  Acc: 0.9791
Val Loss: 0.0467  Acc: 0.9832
Epoch 8/29
----------
Train Loss: 0.0440  Acc: 0.9856
Val Loss: 0.0431  Acc: 0.9888
Epoch 9/29
----------
Train Loss: 0.0441  Acc: 0.9852
Val Loss: 0.0361  Acc: 0.9888
Epoch 10/29
----------
Train Loss: 0.0396  Acc: 0.9880
Val Loss: 0.0363  Acc: 0.9860
Epoch 11/29
----------
Train Loss: 0.0364  Acc: 0.9856
Val Loss: 0.0378  Ac

In [13]:
def get_predictions(model, iterator, device):

    #Para predicción, desactivamos las caractísticas de training
    model.eval()

    images = []
    labels = []
    probs = []

    with torch.no_grad():
        for (data, target) in iterator:
            data = data.to(device)
            #print(target)
            y_pred = model(data)

            top_pred = y_pred.argmax(1, keepdim = True)

            #Almacenamos las etiquetas y las pdf de cada sample
            labels.append(target.cpu())
            probs.append(y_pred.cpu())

    labels = torch.cat(labels, dim = 0)
    probs = torch.cat(probs, dim = 0)

    return labels, probs

In [14]:
model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 2)

model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load("resnet18_finetuned.pth"))
criterion = nn.CrossEntropyLoss()

test_model(model_ft, criterion)


Test Loss: 0.0244  Acc: 0.9888


In [15]:
#Computar predicciones y las etiquetas de máxima probabilidad
labels, probs = get_predictions(model_ft, test_loader, device)

pred_labels = torch.argmax(probs, 1)
from sklearn import metrics

print(metrics.classification_report(labels, pred_labels, digits=3))

              precision    recall  f1-score   support

           0      0.987     0.915     0.949        82
           1      0.989     0.998     0.994       631

    accuracy                          0.989       713
   macro avg      0.988     0.957     0.972       713
weighted avg      0.989     0.989     0.989       713



In [16]:
test_loader

In [17]:
labels

tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Solución 2: Freezing

En esta técnica, el modelo original pre-entrenado se congela, de manera que sus parámetros no cambian. Lo único que se entrena es la última capa, que debe aprender a solucionar el problema usando las características computadas por la red pre-entrenada.

In [18]:
#~10min
#Usamos un modelo pre-entrenado ResNet18 con dataset Imagenet
model_fr = models.resnet18(pretrained=True)
for param in model_fr.parameters():
  param.requires_grad = False

#Vamos a cambiar la última capa de la red. La red original fur entrenada con 
# 1000 clases. Ahora solo necesitamos 6 neuronas de salida.
# Cambiamos la arquitectura final de la red

num_ft = model_fr.fc.in_features
model_fr.fc = nn.Linear(num_ft, 6)

model_fr = model_fr.to(device)
criterion = nn.CrossEntropyLoss()

#La red original fue entrenada con SGD y un lr inicial de 0.1, el cual decrementaba cada vez que el 
# error se estancaba. Nosotros partimos con Adam y un lr bajo, dado que solo queremos tunear la red
# para el nuevo problema

optimizer = torch.optim.SGD(model_fr.parameters(), lr = 0.001, momentum=0.9)

# Empezamos con un número de épocas bajo. A más épocas podemos mejorar el performance
model_fr = train_model(model_fr, criterion, optimizer, num_epochs=30)

#Guardamos la mejor red, en términos de accuracy de validación
torch.save(model_fr.state_dict(), 'resnet18_freezed.pth')

Epoch 0/29
----------
Train Loss: 0.3464  Acc: 0.8994
Val Loss: 0.1309  Acc: 0.9664
Epoch 1/29
----------
Train Loss: 0.1629  Acc: 0.9471
Val Loss: 0.0991  Acc: 0.9748
Epoch 2/29
----------
Train Loss: 0.1481  Acc: 0.9491
Val Loss: 0.0871  Acc: 0.9748
Epoch 3/29
----------
Train Loss: 0.1400  Acc: 0.9479
Val Loss: 0.0824  Acc: 0.9804
Epoch 4/29
----------
Train Loss: 0.1269  Acc: 0.9555
Val Loss: 0.0802  Acc: 0.9720
Epoch 5/29
----------
Train Loss: 0.1186  Acc: 0.9587
Val Loss: 0.0794  Acc: 0.9720
Epoch 6/29
----------
Train Loss: 0.1146  Acc: 0.9595
Val Loss: 0.0723  Acc: 0.9832
Epoch 7/29
----------
Train Loss: 0.1159  Acc: 0.9579
Val Loss: 0.0744  Acc: 0.9832
Epoch 8/29
----------
Train Loss: 0.1177  Acc: 0.9571
Val Loss: 0.0762  Acc: 0.9720
Epoch 9/29
----------
Train Loss: 0.1116  Acc: 0.9587
Val Loss: 0.0712  Acc: 0.9748
Epoch 10/29
----------
Train Loss: 0.1075  Acc: 0.9595
Val Loss: 0.0652  Acc: 0.9804
Epoch 11/29
----------
Train Loss: 0.1135  Acc: 0.9567
Val Loss: 0.0825  Ac

In [ ]:
model_fr = models.resnet18(pretrained=True)
num_ft = model_fr.fc.in_features
model_fr.fc = nn.Linear(num_ft, 6)

model_fr = model_fr.to(device)

model_fr.load_state_dict(torch.load("resnet18_freezed.pth"))
criterion = nn.CrossEntropyLoss()

test_model(model_fr, criterion)

# Solución 3: Transfer learning con schedulers de learning rate

Una estrategia común al entrenar redes neuronales es cambiar el learning rate bajo algún criterio durante el entrenamiento. Estas estrategias responden a la suposición de que el entrenamiento se estanca por momentos y es necesario cambiar las reglas de actualización para seguir mejorando.

In [ ]:
def train_model_scheduler(model, criterio, optimizer, scheduler, num_epochs = 25):
   
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)
    
    #Train model
    scheduler.step()
    model.train()
    
    running_loss = 0.0
    running_corrects = 0.0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
          
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss /len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
      
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    
    #Validation 
    model.eval()
    running_loss = 0.0
    running_corrects = 0.0
    
    for inputs, labels in val_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
        
      with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
          
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
     
    epoch_loss = running_loss /len(test_dataset)
    epoch_acc = running_corrects.double() / len(val_dataset)
      
    print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        
  print('Best val accucary: {:.4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)
  return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 6)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model_ft.parameters(), lr = 0.01, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Empezamos con un número de épocas bajo. A más épocas podemos mejorar el performance
model_ft = train_model_scheduler(model_ft, criterion, optimizer,exp_lr_scheduler, num_epochs=30)

#Guardamos la mejor red, en términos de accuracy de validación
torch.save(model_ft.state_dict(), 'resnet18_scheduler.pth')

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ft = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ft, 6)

model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load("resnet18_scheduler.pth"))
criterion = nn.CrossEntropyLoss()

test_model(model_ft, criterion)